## Participant-level analysis pipeline for calculating ERN in the SART task

<span style='color: blue'>Version 1.1 (2023-07-24)<br></span>Dr. Daniel Holt (APKS) and Dr. Torsten Wüstenberg (CNSR)

**Aim:** Calculate the ERN/CRN for the SART Go/NoGo-Task.

**Gereral analysis steps:**

1) *Select and load data for analysis*
2) *Re-reference and pre-filter*
3) *Ocular artifact corection*
4) *Calculate CRN*
5) *Calculate ERN*
6) *Extract means and save results*


## Preparations 

### Import libraries

The analysis pipeline is based on the following libraries. In case of an error in the execution of this cell, probably one or more of the libraries is not installed. In this case, start a terminal in **ANACONDA.NAVIGATOR** *Environments>Terminal* and install the library in question using the command </br><span style='color: red'>*pip install [library name]*</span>.

In [ ]:
import mne
import numpy as np
import pandas as pd
import ipyfilechooser

mne.set_log_level("ERROR")  # only show errors (no warnings or information messages)
results_csv = "ERN_Results.csv"  # save analysis results to this file
scal = {
    "eeg": 100e-6,
    "misc": 100e-6,
}  # default display scaling, use e-6 for display in microvolts

### Channel information and display configuration

In [ ]:
%matplotlib qt
ch_type_mapping = {"VEOG": "eog", "ECG": "ecg", "EMG": "emg"}
scal = {"eeg": 200e-6, "eog": 1000e-6}  # use e-6 for display in microvolts

## Pipeline

### 1) Select and load data for analysis

Run the cell below and click "Select" to choose the BrainVision **.vhdr** file of the data set to be analyzed. The participant ID defaults to the file name, change the line `participant = ...` for setting a different value.

In [ ]:
fc = ipyfilechooser.FileChooser(".")
display(fc)

In [ ]:
participant = fc.selected_filename[-8:-5]
raw = mne.io.read_raw_brainvision(fc.selected, preload=True)
raw.set_channel_types(ch_type_mapping)

### 2) Re-reference and pre-filter

Re-reference from recording reference Cz to TP9 and TP10, apply a basic filter.

In [ ]:
raw.add_reference_channels("Cz")
raw.set_eeg_reference(["TP9", "TP10"])
raw.drop_channels(["TP9", "TP10"])
raw.filter(l_freq=0.1, h_freq=None)
raw.plot(scalings=scal)

### 3) Simple ICA-based ocular artifact reduction


Fit ICA in data segment with stimulus events (markers 11 to 189), select component with highest correlation with VEOG for correction. This processing step may take a while.

In [ ]:
events, _ = mne.events_from_annotations(raw)
events = [event for event in events if event[2] in range(11, 189) and event[0] > 5000]
raw_ica = raw.copy().filter(l_freq=1.0, h_freq=None)
raw_ica.crop(events[0][0] / 500.0, events[-1][0] / 500.0)

ica = mne.preprocessing.ICA(n_components=15, max_iter="auto", random_state=999)
ica.fit(raw_ica)

_, scores_eog = ica.find_bads_eog(raw_ica, "VEOG")
eog_maxcorr_idx = np.argmax(abs(scores_eog))
ica.exclude = [eog_maxcorr_idx]
clean = raw.copy()
ica.apply(clean)

ica.plot_components(
    title=f"Component selected for ocular artifact correction: {ica._ica_names[eog_maxcorr_idx]}"
)

### 4) Calculate CRN

Calculate correct-response related negativity for go-trials. Marker code of correct go-trials ends with "2". Reject epochs with more than 200 uV amplitude difference. Baseline is set to -.4 to -.2. Mark bad epochs for exclusion in the plot.

In [ ]:
clean.filter(l_freq=0.1, h_freq=30)

go_events, _ = mne.events_from_annotations(clean, regexp=".*2$")
go_epochs = mne.Epochs(clean, go_events, tmin=-0.4, tmax=0.8, detrend=0)
go_epochs.drop_bad(reject={"eeg": 200e-6})
go_epochs.apply_baseline(((-0.4, -0.2)))
go_averages = go_epochs.average(picks=["Fz", "FCz", "Cz"])

go_epochs.plot(scalings=50e-6)
go_averages.plot_joint(times=[0, 0.05, 0.1], show=False)

In [ ]:
go_epochs_marked = len(
    [drop for drop in go_epochs.drop_log if drop and drop[0] == "USER"]
)
print(
    f"{go_epochs_marked} von {len(go_epochs) + go_epochs_marked} Go-Epochen manuell ausgeschlossen."
)

### 5) Calculate ERN

Calculate error-related negativity for nogo-trials with errors. Marker code of error-trials ends with "6".

In [ ]:
nogo_events, _ = mne.events_from_annotations(clean, regexp=".*6$")
nogo_epochs = mne.Epochs(clean, nogo_events, tmin=-0.4, tmax=0.8, detrend=0)
nogo_epochs.drop_bad(reject={"eeg": 200e-6})
nogo_epochs.apply_baseline(((-0.4, -0.2)))
nogo_averages = nogo_epochs.average(picks=["Fz", "FCz", "Cz"])

nogo_epochs.plot(scalings=50e-6)
nogo_averages.plot_joint(times=[0, 0.05, 0.1], show=False)
nogo_epochs_marked = len(
    [drop for drop in nogo_epochs.drop_log if drop and drop[0] == "USER"]
)

In [ ]:
nogo_epochs_marked = len(
    [drop for drop in go_epochs.drop_log if drop and drop[0] == "USER"]
)
print(
    f"{nogo_epochs_marked} von {len(nogo_epochs) + nogo_epochs_marked} Go-Epochen manuell ausgeschlossen."
)

### 6) Extract and save mean ERPs

Calcuate mean amplitude at FCz for 0 to 100 msec in each epoch.

In [ ]:
erp_analysis = {
    "PID": participant,
    "ERN": (np.mean(nogo_averages.get_data("FCz")[0, 200:250]) * 1e6).round(3),
    "ERN_n": len(nogo_epochs),
    "CRN": (np.mean(go_averages.get_data("FCz")[0, 200:250]) * 1e6).round(3),
    "CRN_n": len(go_epochs),
}

In [ ]:
results = pd.read_csv(results_csv)
erp_analysis_df = pd.DataFrame(erp_analysis, index=[0])
results = pd.concat([results, erp_analysis_df])
results.to_csv(results_csv, index=False)
print(f'Analyse-Ergebnis in "{results_csv}" gespeichert.')
results